In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.optimize import minimize

def x_model(t, theta_rad, M, X):
    """Calculates the predicted x-values."""
    term1 = t * np.cos(theta_rad)
    term2 = np.exp(M * np.abs(t)) * np.sin(0.3 * t) * np.sin(theta_rad)
    return term1 - term2 + X

def y_model(t, theta_rad, M):
    """Calculates the predicted y-values. Note: X is not in this equation."""
    term1 = t * np.sin(theta_rad)
    term2 = np.exp(M * np.abs(t)) * np.sin(0.3 * t) * np.cos(theta_rad)
    return 42 + term1 + term2

def l1_loss(params, t_values, x_data, y_data):
    """
    Calculates the L1 loss (Mean Absolute Error) between
    predicted and actual data.
    """
    theta_rad, M, X = params

    # Calculate predicted values
    x_pred = x_model(t_values, theta_rad, M, X)
    y_pred = y_model(t_values, theta_rad, M)

    # Calculate L1 loss
    x_loss = np.sum(np.abs(x_data - x_pred))
    y_loss = np.sum(np.abs(y_data - y_pred))

    total_loss = x_loss + y_loss

    # Optional: Print loss during optimization to see progress
    # print(f"Loss: {total_loss:.2f}")

    return total_loss

def get_initial_guesses(x_data, y_data):
    """
    Uses linear regression on the underlying structure to get
    excellent initial guesses for theta and X.
    """
    # Fit x = A*y + B
    lin_reg = stats.linregress(y_data, x_data)
    A = lin_reg.slope
    B = lin_reg.intercept

    # From our analysis:
    # A = cot(theta) => theta = arccot(A) = arctan(1/A)
    theta_guess_rad = np.arctan(1 / A)

    # X = B + 42*A
    X_guess = B + 42 * A

    # M is small, so 0 is a great starting guess
    M_guess = 0.0

    print("--- Initial Guesses (from Linear Regression) ---")
    print(f"Regression: x = {A:.4f} * y + {B:.4f}")
    print(f"Initial Theta (deg): {np.rad2deg(theta_guess_rad):.4f}")
    print(f"Initial X: {X_guess:.4f}")
    print(f"Initial M: {M_guess}")
    print("--------------------------------------------------\n")

    return [theta_guess_rad, M_guess, X_guess]

def main():
    # 1. Load Data
    try:
        data = pd.read_csv('xy_data.csv')
        x_data = data['x'].values
        y_data = data['y'].values
    except FileNotFoundError:
        print("Error: xy_data.csv not found.")
        print("Please make sure the file is in the same directory.")
        return

    # 2. Create corresponding 't' values
    N = len(data)
    # Assume data points correspond to uniformly spaced t values
    t_values = np.linspace(6, 60, N)

    # 3. Get Initial Guesses
    initial_params = get_initial_guesses(x_data, y_data)

    # 4. Define Parameter Bounds (slightly inset from hard limits)
    # Bounds: [(theta_rad_min, max), (M_min, max), (X_min, max)]
    theta_min_rad = np.deg2rad(0.01)
    theta_max_rad = np.deg2rad(49.99)
    bounds = [
        (theta_min_rad, theta_max_rad),
        (-0.0499, 0.0499),
        (0.01, 99.99)
    ]

    # 5. Run the Optimization
    print("Starting optimization... (This may take a moment)")
    result = minimize(
        l1_loss,
        initial_params,
        args=(t_values, x_data, y_data),
        method='L-BFGS-B',  # A good method for bound-constrained problems
        bounds=bounds,
        options={'disp': True, 'ftol': 1e-9, 'gtol': 1e-7}
    )

    # 6. Show Results
    if result.success:
        final_theta_rad, final_M, final_X = result.x
        final_theta_deg = np.rad2deg(final_theta_rad)

        print("\n--- Optimization Successful ---")
        print(f"Final Theta (deg): {final_theta_deg:.6f}")
        print(f"Final Theta (rad): {final_theta_rad:.6f}")
        print(f"Final M:           {final_M:.6f}")
        print(f"Final X:           {final_X:.6f}")

        # 7. Format for Submission
        # Using 4-5 decimal places for a clean submission string
        theta_str = f"{final_theta_rad:.4f}"
        M_str = f"{final_M:.4f}"
        X_str = f"{final_X:.4f}"

        latex_string = (
            f"\\left(t*\\cos({theta_str})-e^{{{M_str}\\left|t\\right|}}\\cdot"
            f"\\sin(0.3t)\\sin({theta_str})\\ +{X_str},42+\\ t*\\sin({theta_str})"
            f"+e^{{{M_str}\\left|t\\right|}}\\cdot\\sin(0.3t)\\cos({theta_str})\\right)"
        )

        print("\n--- Required Submission String ---")
        print(latex_string)

    else:
        print("\n--- Optimization Failed ---")
        print(f"Message: {result.message}")

if __name__ == "__main__":
    main()